# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [1]:
import random
import numpy
print("numpy", numpy.__version__)
import pandas
print("pandas", pandas.__version__)
import seaborn
print("seaborn", seaborn.__version__)
import matplotlib
print("matplotlib", matplotlib.__version__)
from matplotlib import pyplot as plt
import sys
print(sys.version_info)

numpy 1.23.5
pandas 1.5.2
seaborn 0.12.2
matplotlib 3.6.2
sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)


In [ ]:
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

In [ ]:
import lib_simulation

### global variables for simulation

#### user-defined

In [ ]:
import configuration_simCount1_skills2_levels2_duration1_people8_social0_ticks100 as config

#import configuration_simCount1_skills2_levels2_duration1_people8_social2_ticks100 as config

### input validation

In [ ]:
if len(config.skill_set_for_people)<1:
    raise Exception("invalid value")
if len(config.skill_set_for_tasks)<1:
    raise Exception("invalid value")
    
if config.max_skill_level_per_person<1:
    raise Exception("invalid value")
if config.max_skill_level_per_task<1:
    raise Exception("invalid value")
if config.max_task_duration_in_ticks<1:
    raise Exception("invalid value")
    
if config.min_number_of_people<1:
    raise Exception("invalid value")
if config.max_number_of_people<1:
    raise Exception("invalid value")
if ((config.min_number_of_people) > (config.max_number_of_people)):
    print(config.min_number_of_people)
    print(config.max_number_of_people)
    print((config.min_number_of_people) > (config.max_number_of_people))
    raise Exception("invalid value")

if config.min_social_circle_size<0:
    raise Exception("invalid value")
if config.max_social_circle_size<0:
    raise Exception("invalid value")
if config.min_social_circle_size>config.max_social_circle_size:
    print(config.min_social_circle_size)
    print(config.max_social_circle_size)
    raise Exception("invalid value")
    
if config.max_ticks_to_simulate<1:
    raise Exception("invalid value")

## single instance

In [ ]:
number_of_people = config.min_number_of_people
social_circle_size = config.min_social_circle_size

In [ ]:
list_of_people = []
for person_index in range(number_of_people):
    list_of_people.append(lib_simulation.CreatePerson(person_index, 
                                                      config.skill_set_for_people, 
                                                      config.max_skill_level_per_person))

In [ ]:
lib_simulation.check_population_for_capability(list_of_people,
                                               config.skill_set_for_people,
                                               config.max_skill_level_per_person)

instead of a static list of tasks to draw from,   
create an infinite backlog of work

### Assessment tools

In [ ]:
lib_simulation.show_all_people(list_of_people)

In [ ]:
lib_simulation.current_status_of_people(list_of_people)

In [ ]:
lib_simulation.cumulative_task_backlog_size(list_of_people)

### Initialize Simulation

In [ ]:
tasks_dict = {} # track the generated tasks for post-simulation analysis

task_id = -1

# randomly distribute tasks to people
for index in range(len(list_of_people)):
    #print('person ID',list_of_people[index].unique_id)
    task_id+=1
    tasks_dict = lib_simulation.new_task(task_id,
                                         config.skill_set_for_tasks,
                                         config.max_skill_level_per_task,
                                         config.max_task_duration_in_ticks,
                                         tasks_dict)
    list_of_people[index].backlog_of_tasks.append(tasks_dict[task_id])

In [ ]:
lib_simulation.current_status_of_people(list_of_people)

### Begin ticks of Simulation

In [ ]:
list_of_people, tasks_dict = lib_simulation.simulate(config.skill_set_for_tasks,
                                                     config.max_skill_level_per_task,
                                                     config.max_ticks_to_simulate,
                                                     config.max_task_duration_in_ticks,
                                                     list_of_people,
                                                     tasks_dict)

## Post-simulation analysis

### a single person

In [ ]:
list_of_dicts = []
for tick_index, journal_dict in list_of_people[0].work_journal_per_tick.items():
    #print(tick_index)
    this_dict = {'tick': tick_index}
    for k,v in journal_dict.items():
        if k == "task":
            for task_k, task_v in v.items():
                this_dict[task_k] = task_v
        else:    
            this_dict[k] = v
    #print(this_dict)
    list_of_dicts.append(this_dict)

In [ ]:
list_of_people[0].skill_specialization_dict

In [ ]:
df_a_persons_journal = pandas.DataFrame(list_of_dicts)
df_a_persons_journal

# Visualization of the Simulation

In [ ]:
for index,person in enumerate(list_of_people):
    print(index,person.skill_specialization_dict)

In [ ]:
# TODO: A heat map of the ensemble of people for specializations and skill levels

In [ ]:
aggregate_person_dict = {}
for specialization in config.skill_set_for_people:
    aggregate_person_dict[specialization] = [0 for _ in range(config.max_skill_level_per_person)]

#aggregate_person_dict

In [ ]:
for person_id, person in enumerate(list_of_people):
    for specialization, skilllevel in person.skill_specialization_dict.items():
        #print(person_id, specialization, skilllevel)
        if skilllevel>0:
            aggregate_person_dict[specialization][skilllevel-1] += 1
        #print(aggregate_person_dict)
        
#aggregate_person_dict

In [ ]:
df_cumulative_people = pandas.DataFrame(aggregate_person_dict)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
df_cumulative_people.set_index(pandas.Index(list(range(1,config.max_skill_level_per_person+1))))
df_cumulative_people

In [ ]:
# https://stackoverflow.com/a/34712544/1164295
seaborn.set(font_scale=1.5)
# https://stackoverflow.com/a/57266083/1164295
seaborn.heatmap(df_cumulative_people, 
                annot=True, 
                fmt="d",
                yticklabels=df_cumulative_people.index, 
                cmap='Blues', 
                linewidths=1, 
                linecolor='black');
plt.title("Cumulative counts across all people")
plt.gca().set_xlabel('Specialization', fontsize=15);
plt.gca().set_ylabel('Skill-level', fontsize=15);

In [ ]:
# TODO: A heat map of the ensemble of task per specializations in skill levels

In [ ]:
aggregate_task_dict = {}
for specialization in config.skill_set_for_tasks:
    aggregate_task_dict[specialization] = [0 for _ in range(config.max_skill_level_per_task)]

aggregate_task_dict

In [ ]:
for task_id,this_task in tasks_dict.items():
    #print('specialization:',task['specialization'])
    #print('skill-level:',task['skill level'])
    if this_task['skill level']>0:
        aggregate_task_dict[this_task['specialization']][this_task['skill level']-1] += 1

In [ ]:
df_cumulative_tasks = pandas.DataFrame(aggregate_task_dict)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
df_cumulative_tasks = df_cumulative_tasks.set_index(pandas.Index(list(range(1,config.max_skill_level_per_task+1))))

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

# https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html

# https://stackoverflow.com/a/34712544/1164295
seaborn.set(font_scale=1.5)
# https://stackoverflow.com/a/57266083/1164295
seaborn.heatmap(df_cumulative_tasks, 
                annot=True, 
                fmt="d",
                yticklabels=df_cumulative_tasks.index, 
                cmap='Blues', 
                linewidths=1, 
                linecolor='black');
plt.title("Cumulative counts across all tasks")
plt.gca().set_xlabel('Specialization', fontsize=15);
plt.gca().set_ylabel('Skill-level', fontsize=15);

In [ ]:
# TODO: Heat map with axes of time and person index, 
# with the color being the length of the backlog per person per time

In [ ]:
person_time_backlog_dict = {}
for person_index in range(len(list_of_people)):
    
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","
    
    person_label = str(person_index)+": "+description[:-1]
    
    person_time_backlog_dict[person_label] = []
    for tick_index, journal_dict in list_of_people[person_index].work_journal_per_tick.items():
        #print(person_id, tick_index, journal_dict['number of tasks in backlog'])
        person_time_backlog_dict[person_label].append(journal_dict['number of tasks in backlog'])

In [ ]:
df_person_time_backlog = pandas.DataFrame(person_time_backlog_dict)
#df_person_time_backlog

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_backlog)
plt.title('backlog length per person')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

## task ID per person versus time

In [ ]:
person_time_taskID_dict = {}
for person_id, person in enumerate(list_of_people):
    person_time_taskID_dict[person_id] = []
    for tick, work_dict in person.work_journal_per_tick.items():
        #print(work_dict)
        person_time_taskID_dict[person_id].append(work_dict['task']['task ID'])

In [ ]:
df_person_time_taskID = pandas.DataFrame(person_time_taskID_dict)
df_person_time_taskID

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_taskID)
plt.title('task ID')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

# ratio of the elapsed time for a task versus the number task for task.

In [ ]:
min(df_person_time_taskID.min())

In [ ]:
elapsed_task_timing = {}
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html
for task_id in range(min(df_person_time_taskID.min()),
                     max(df_person_time_taskID.max())):
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
    list_of_ticks_containing_task = list(df_person_time_taskID[
        df_person_time_taskID==task_id].dropna(axis=0,how='all').index)
    
    #assert(task_id==list_of_tasks[task_id-1]['task ID'])
    
    elapsed_task_timing[task_id] = {'first seen': min(list_of_ticks_containing_task),
                                    'last seen': max(list_of_ticks_containing_task),
                                    'task duration': 1}

In [ ]:
df_task_timing = pandas.DataFrame(elapsed_task_timing).T
df_task_timing['task actual duration'] = df_task_timing['last seen'] - df_task_timing['first seen']+1
df_task_timing['ratio of actual to minimum'] = df_task_timing['task actual duration']/df_task_timing['task duration']
#df_task_timing

In [ ]:
print("when the temporal cost of coordination is the same as just doing the work,")
print("with",number_of_people,"people the workload took",
      round(sum(df_task_timing['task actual duration'])/sum(df_task_timing['task duration']),2))
print("longer than if one qualified person had been assigned the same workload")

In [ ]:
# TODO: Heat map with axes of time and person index, 
# with the color being the status (working, idle, coordinating)

In [ ]:
# TODO: A pie chart of the named persona types like unicorn and specialist. 
# The biggest piece of pie will be the others

## task throughput vs time

In [ ]:
task_count_per_time_dict = {}
for person_id, person in enumerate(list_of_people):
    for tick, work_dict in person.work_journal_per_tick.items():
        if tick not in task_count_per_time_dict.keys():
            task_count_per_time_dict[tick] = 0
            
        if "task from" in work_dict.keys():
            #print(person_id, tick, work_dict["task from"])
            task_count_per_time_dict[tick] += 1

In [ ]:
plt.scatter(x=task_count_per_time_dict.keys(), 
            y=task_count_per_time_dict.values())
plt.ylabel('number of tasks\nstarted during this tick')
plt.xlabel('time [ticks]');

In [ ]:
plt.scatter(x=list(task_count_per_time_dict.keys()), 
            y=list(numpy.cumsum(list(task_count_per_time_dict.values()))),
            label="number of people="+str(number_of_people))

plt.ylabel('cumulative task count')
plt.xlabel('time [ticks]');
plt.legend();

# The reward for good work is more work

In [ ]:
people_versus_time_task_count={}
for person_id, person in enumerate(list_of_people):
    people_versus_time_task_count[person_id] = []
    completed_task_count = 0
    for tick, work_dict in person.work_journal_per_tick.items():
        if work_dict['outcome']=='task completed':
            completed_task_count+=1
        people_versus_time_task_count[person_id].append(completed_task_count)

In [ ]:
list_of_labels = []
list_of_task_counts = []

for person_index, completed_task_count in people_versus_time_task_count.items():
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","

    person_label = "person "+str(person_index)+": "+description[:-1]
    
    # these two lists are not used in this cell; they are for the stacked plot
    list_of_labels.append(person_label)
    list_of_task_counts.append(completed_task_count)
        
    plt.scatter(x=range(len(completed_task_count)),
                y=completed_task_count, 
                label=person_label)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('cumulative count\nof completed tasks');
plt.title('social circle size ='+str(social_circle_size));

## work done per person

In [ ]:
# https://www.python-graph-gallery.com/251-stacked-area-chart-with-seaborn-style
# https://www.python-graph-gallery.com/250-basic-stacked-area-chart

# set seaborn style
seaborn.set_theme()
    
plt.stackplot(range(len(completed_task_count)),
              list_of_task_counts,
              labels=list_of_labels)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('cumulative count\nof completed tasks');
plt.title('social circle size ='+str(social_circle_size));

## percentage stacked area chart

https://www.python-graph-gallery.com/255-percentage-stacked-area-chart

In [ ]:
task_dict = {}
for person_index, cumulative_task_count in enumerate(list_of_task_counts):
    task_dict[person_index] = cumulative_task_count
    
df_task_cumulative_count = pandas.DataFrame(task_dict)

In [ ]:
# https://stackoverflow.com/a/46738366

df_task_cumulative_percent = df_task_cumulative_count.divide(df_task_cumulative_count.sum(axis=1), axis=0)
  
#plot_me = df_task_cumulative_percent.T

#plot_me = df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]]

#plot_me[plot_me.columns[::-1]]

#df_to_plot = df_task_cumulative_percent.T


# https://stackoverflow.com/a/27817770

In [ ]:
plt.stackplot(range(len(completed_task_count)),
              #df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]].T,
              df_task_cumulative_percent.T,
              #plot_me.T,
              #df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]].T,
              labels=list_of_labels)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('percent of\ncompleted tasks');
plt.title('social circle size ='+str(social_circle_size));

# TODO: https://en.wikipedia.org/wiki/Little%27s_law